importing all libs

In [3]:
import pandas as pd
import numpy as np

In [31]:
df = pd.read_csv("./dataset.csv")

Identifying unique channel names before standardization

In [38]:
print(df["Channel"].unique())

['ARY News' 'Hum News' 'Samaa TV' 'Dawn News' 'Geo News' 'AbbTakk News'
 'Express News']


Correct Channel names

In [47]:
df['Channel'] = df['Channel'].replace({
    "geo": "Geo News",
    "Geo": "Geo News",
    "GEO NEWS": "Geo News",
    "ARY": "ARY News",
    "ary": "ARY News",
    "ARYNEWS": "ARY News",
    "Samaa": "Samaa TV",
    "SAMAA TV": "Samaa TV",
    "Express": "Express News",
    "Express-News": "Express News",
    "hum news": "Hum News",
    "HUM News": "Hum News",
    "DawnNews": "Dawn News",
    "DAWN": "Dawn News"
})

print(df['Channel'].unique())



['ARY News' 'Hum News' 'Samaa TV' 'Dawn News' 'Geo News' 'AbbTakk News'
 'Express News']


Find Unique Names

In [ ]:
print(df["Journalist"].unique())

['Najam Sethi' 'Dr. Shahid Masood' 'Kamran Shahid' 'Matiullah Jan'
 'RAUF KLASSRA' 'Ansar Abbasi' 'Umar Cheema' 'Shahzeb Khanzada'
 'shahzeb khanzada' 'Kamran Khan' 'K. Khan' 'Asma Shirazi'
 'Mohsin Raza Khan' 'Saleem Safi' 'Talat Hussain' 'Rauf Klasra'
 'Owais Tohid' 'Nusrat Javed' 'Mohsin Raza']


Makes Journalist Name Consistant and fix spelling mistakes

In [44]:
df["Journalist"] = df["Journalist"].str.title()
df["Journalist"] = df["Journalist"].replace({
    "Rauf Klassra": "Rauf Klasra",
    "Mohsin Raza Khan": "Mohsin Raza"
})
print(df["Journalist"].unique())

['Najam Sethi' 'Dr. Shahid Masood' 'Kamran Shahid' 'Matiullah Jan'
 'Rauf Klasra' 'Ansar Abbasi' 'Umar Cheema' 'Shahzeb Khanzada'
 'Kamran Khan' 'K. Khan' 'Asma Shirazi' 'Mohsin Raza' 'Saleem Safi'
 'Talat Hussain' 'Owais Tohid' 'Nusrat Javed']
